In [ ]:
#!cp drive/MyDrive/data.csv data.csv

# **K-Means**

Библиотеки подключаю необходимые (KMeans буду использовать в реализации sklearn)

In [ ]:
from sklearn.cluster import KMeans

import numpy as np

import pandas as pd

Подгружаю датасет

In [ ]:
data = pd.read_csv('data.csv')

Посмотреть можно на него

In [ ]:
data

,Unnamed: 0,LotArea,GarageArea,MSZoning,LotFrontage,Condition1,HouseStyle,SalePrice,YearBuilt,YrSold,Neighborhood
0,479,5925,672,RM,50.0,Norm,1Story,89471,1937,2007,OldTown
1,793,9158,474,RL,76.0,Norm,1Story,225000,2007,2007,Somerst
2,339,12400,297,RL,66.0,Feedr,1Story,155000,1958,2009,NAmes
3,142,8520,720,RL,71.0,Artery,1.5Fin,166000,1952,2010,NAmes
4,1453,17217,0,RL,90.0,Norm,1Story,84500,2006,2006,Mitchel
...,...,...,...,...,...,...,...,...,...,...,...
1455,831,3180,480,FV,30.0,Norm,2Story,151000,2005,2006,Somerst
1456,522,5000,420,RM,50.0,Feedr,1.5Fin,159000,1947,2006,BrkSide
1457,1217,8640,552,FV,72.0,Norm,1Story,229456,2009,2009,Somerst
1458,162,12182,532,RL,95.0,Norm,1Story,220000,2005,2010,NridgHt


Выберу колонки для анализа LotArea, GarageArea, LotFrontage, SalePrice

Выбираю их для анализа, потому что с одной стороны должна проступать зависимость между площадью участка/гаража и стоимостью, и с другой стороны интересна зависимость от LotFrontage. С одной стороны, она должна быть выше у очень дорогих участков, поскольку у них часто отдельная территория, которая может быть достаточно далеко от дороги и с другой стороны большое расстояние до дороги может быть большим минусом для обычных людей, что может повлиять на стоимость в отрицательную сторону

В датасете у LotFrontage иногда присутствует NaN, заменю его на 0

In [ ]:
data_chosen = data[['LotArea', 'GarageArea', 'LotFrontage', 'SalePrice']]

data_chosen = data_chosen.fillna(0)

In [ ]:
data_chosen

,LotArea,GarageArea,LotFrontage,SalePrice
0,5925,672,50.0,89471
1,9158,474,76.0,225000
2,12400,297,66.0,155000
3,8520,720,71.0,166000
4,17217,0,90.0,84500
...,...,...,...,...
1455,3180,480,30.0,151000
1456,5000,420,50.0,159000
1457,8640,552,72.0,229456
1458,12182,532,95.0,220000


## **K-means с K = 5**

Инициализируем KMeans 100 рандомными инициализациями на 5 кластеров

In [ ]:
KMeans_5 = KMeans(n_clusters=5, init='random', n_init=100)

Применим алгоритм KMeans

In [ ]:
labels_5 = KMeans_5.fit_predict(data_chosen)

 Сохраним номера кластеров, предсказанных алгоритмом

In [ ]:
data_chosen['Label_5'] = labels_5
data['Label_5'] = labels_5

In [ ]:
data_chosen

,LotArea,GarageArea,LotFrontage,SalePrice,Label_5
0,5925,672,50.0,89471,1
1,9158,474,76.0,225000,3
2,12400,297,66.0,155000,2
3,8520,720,71.0,166000,2
4,17217,0,90.0,84500,1
...,...,...,...,...,...
1455,3180,480,30.0,151000,2
1456,5000,420,50.0,159000,2
1457,8640,552,72.0,229456,3
1458,12182,532,95.0,220000,3


Посчитаю средние фичи глобально по датасету

In [ ]:
mean_LotArea = np.mean(data_chosen['LotArea'])
mean_GarageArea = np.mean(data_chosen['GarageArea'])
mean_LotFrontage = np.mean(data_chosen['LotFrontage'])
mean_SalePrice = np.mean(data_chosen['SalePrice'])

Посчитаю средние фичи по каждому кластеру и относительную разницу фичи в кластере и во всем датасете

In [ ]:
rds = dict()
for label in range(5):
  rds[label] = dict()

for label in range(5):
  label_data = data_chosen[labels_5 == label]
  mean_cluster_LotArea = np.mean(label_data['LotArea'])
  mean_cluster_GarageArea = np.mean(label_data['GarageArea'])
  mean_cluster_LotFrontage = np.mean(label_data['LotFrontage'])
  mean_cluster_SalePrice = np.mean(label_data['SalePrice'])

  rds[label]['LotArea'] = 100 * (mean_cluster_LotArea / mean_LotArea - 1)
  rds[label]['GarageArea'] = 100 * (mean_cluster_GarageArea / mean_GarageArea - 1)
  rds[label]['LotFrontage'] = 100 * (mean_cluster_LotFrontage / mean_LotFrontage - 1)
  rds[label]['SalePrice'] = 100 * (mean_cluster_SalePrice / mean_SalePrice - 1)


Посмотрим на относительную разницу по номеру кластера и фиче

In [ ]:
rds

{0: {'GarageArea': 78.91465265808364,
  'LotArea': 93.02471015430199,
  'LotFrontage': 58.856714425477065,
  'SalePrice': 213.03505730407696},
 1: {'GarageArea': -28.033438059058792,
  'LotArea': -20.770610492050835,
  'LotFrontage': -5.714422028521515,
  'SalePrice': -34.91775778136075},
 2: {'GarageArea': 2.4370801997449565,
  'LotArea': -4.462017313302502,
  'LotFrontage': -6.483426889639277,
  'SalePrice': -3.553135348912606},
 3: {'GarageArea': 29.218626929234293,
  'LotArea': 27.826044583950104,
  'LotFrontage': 7.201856570884946,
  'SalePrice': 35.26458090958269},
 4: {'GarageArea': 59.28540980916139,
  'LotArea': 53.67483732066223,
  'LotFrontage': 37.11419797608648,
  'SalePrice': 94.10431316898395}}

Для каждого кластера достанем фичи, которые сильно отклоняют его от среднего

Для лимита сильного отклонения возьму 25

Для очень сильного отклонения - 70

In [ ]:
limit = 25
limit_strong = 70

V_k_plus = dict()
V_k_minus = dict()

V_k_strong_plus = dict()
V_k_strong_minus = dict()

for label in range(5):
    V_k_plus[label] = []
    V_k_minus[label] = []
    V_k_strong_plus[label] = []
    V_k_strong_minus[label] = []

for label in range(5):
    for feature in ['LotArea', 'GarageArea', 'LotFrontage', 'SalePrice']:
        if rds[label][feature] > limit_strong:
            V_k_strong_plus[label] += [feature]
        elif rds[label][feature] > limit:
            V_k_plus[label] += [feature]
        elif rds[label][feature] < -limit_strong:
            V_k_strong_minus[label] += [feature]
        elif rds[label][feature] < -limit:
            V_k_minus[label] += [feature]

Фичи сильного положительного отклонения

In [ ]:
V_k_plus

{0: ['LotFrontage'],
 1: [],
 2: [],
 3: ['LotArea', 'GarageArea', 'SalePrice'],
 4: ['LotArea', 'GarageArea', 'LotFrontage']}

Фичи очень сильного положительного отклонения

In [ ]:
V_k_strong_plus

{0: ['LotArea', 'GarageArea', 'SalePrice'],
 1: [],
 2: [],
 3: [],
 4: ['SalePrice']}

Фичи сильного отрицательного отклонения

In [ ]:
V_k_minus

{0: [], 1: ['GarageArea', 'SalePrice'], 2: [], 3: [], 4: []}

Фичи очень сильного отрицательного отклонения (таких не оказалось)

In [ ]:
V_k_strong_minus

{0: [], 1: [], 2: [], 3: [], 4: []}

### Описание на низком уровне

Кластер 0 имеет LotArea, GarageArea, SalePrice очень сильно выше чем в среднем по датасету и LotFrontage просто сильно выше, чем в среднем по датасету

Кластер 1 имеет сильно меньший GarageArea и SalePrice, чем в среднем по датасету

Кластер 2 имеет все примерно в среднем по датасету

Кластер 3 имеет сильно больший LotArea, GarageArea, SalePrice

Кластер 4 имеет сильно больший LotArea, GarageArea, LotFrontage, чем в среднем по датасету и очень сильно больший SalePrice

### Описание на более высоком уровне
Кластера 0, 3 и 4 отвечают за достаточно большие и дорогие участки.

Среди них кластер 0 отвечает за самые дорогие участки, где площадь и стоимость очень сильно выше средних по датасету (их можно назвать особняками). 

Кластер 4 отвечает за участки следующего уровня (которые все еще очень дорогие, но немного поменьше)

Кластер 3 отвечает еще за следующий уровень (при этом эти остаются выше среднего)

Кластер 2 отвечает за средние участки по размерам и стоимости

Кластер 1 отвечает за более дешевые и маленькие участки

Получается примерно такая иерархия по этим кластерам

## **K-means с K = 9**

Возьмем теперь KMeans с 9 кластерами, опять же проинициализируем их 100 рандомными инициализациями

In [ ]:
KMeans_9 = KMeans(n_clusters=9, init='random', n_init=100)

Применим алгоритм и сохраним метки кластеров

In [ ]:
labels_9 = KMeans_9.fit_predict(data_chosen)

In [ ]:
data_chosen['Label_9'] = labels_9
data['Label_9'] = labels_9

In [ ]:
data_chosen

,LotArea,GarageArea,LotFrontage,SalePrice,Label_5,Label_9
0,5925,672,50.0,89471,1,5
1,9158,474,76.0,225000,3,0
2,12400,297,66.0,155000,2,3
3,8520,720,71.0,166000,2,3
4,17217,0,90.0,84500,1,5
...,...,...,...,...,...,...
1455,3180,480,30.0,151000,2,3
1456,5000,420,50.0,159000,2,3
1457,8640,552,72.0,229456,3,0
1458,12182,532,95.0,220000,3,6


Опять же посчитаем средние фичи по датасету

In [ ]:
mean_LotArea = np.mean(data_chosen['LotArea'])
mean_GarageArea = np.mean(data_chosen['GarageArea'])
mean_LotFrontage = np.mean(data_chosen['LotFrontage'])
mean_SalePrice = np.mean(data_chosen['SalePrice'])

А также по каждому кластеру и относительную разницу

In [ ]:
rds = dict()
for label in range(9):
  rds[label] = dict()

for label in range(9):
  label_data = data_chosen[labels_9 == label]
  mean_cluster_LotArea = np.mean(label_data['LotArea'])
  mean_cluster_GarageArea = np.mean(label_data['GarageArea'])
  mean_cluster_LotFrontage = np.mean(label_data['LotFrontage'])
  mean_cluster_SalePrice = np.mean(label_data['SalePrice'])

  rds[label]['LotArea'] = 100 * (mean_cluster_LotArea / mean_LotArea - 1)
  rds[label]['GarageArea'] = 100 * (mean_cluster_GarageArea / mean_GarageArea - 1)
  rds[label]['LotFrontage'] = 100 * (mean_cluster_LotFrontage / mean_LotFrontage - 1)
  rds[label]['SalePrice'] = 100 * (mean_cluster_SalePrice / mean_SalePrice - 1)


Относительная разница фичей по кластерам

In [ ]:
rds

{0: {'GarageArea': 31.124572327684064,
  'LotArea': 18.257279433775153,
  'LotFrontage': 11.2239715591671,
  'SalePrice': 37.09761987165665},
 1: {'GarageArea': 69.21717716590217,
  'LotArea': 41.323288320120334,
  'LotFrontage': 38.320024204964184,
  'SalePrice': 124.39944479298397},
 2: {'GarageArea': 72.33810391991322,
  'LotArea': 119.55170151633689,
  'LotFrontage': 79.18102935932487,
  'SalePrice': 243.3242699635071},
 3: {'GarageArea': -1.0480907720341492,
  'LotArea': -5.353865594847407,
  'LotFrontage': -4.898425949879925,
  'SalePrice': -7.907613456332285},
 4: {'GarageArea': 14.698262691676645,
  'LotArea': 1454.7796229252674,
  'LotFrontage': -34.92214430048733,
  'SalePrice': 63.46205238392106},
 5: {'GarageArea': -41.57719083321971,
  'LotArea': -29.009733884817003,
  'LotFrontage': -10.021647594451577,
  'SalePrice': -49.1034027991892},
 6: {'GarageArea': 11.185093611442664,
  'LotArea': 2.863543819818015,
  'LotFrontage': -6.505786741531228,
  'SalePrice': 11.0175072661

С теми же лимитами считаю, какие фичи сильно отклоняются от средних

In [ ]:
limit = 25
limit_strong = 70

V_k_plus = dict()
V_k_minus = dict()

V_k_strong_plus = dict()
V_k_strong_minus = dict()

for label in range(9):
    V_k_plus[label] = []
    V_k_minus[label] = []
    V_k_strong_plus[label] = []
    V_k_strong_minus[label] = []

for label in range(9):
    for feature in ['LotArea', 'GarageArea', 'LotFrontage', 'SalePrice']:
        if rds[label][feature] > limit_strong:
            V_k_strong_plus[label] += [feature]
        elif rds[label][feature] > limit:
            V_k_plus[label] += [feature]
        elif rds[label][feature] < -limit_strong:
            V_k_strong_minus[label] += [feature]
        elif rds[label][feature] < -limit:
            V_k_minus[label] += [feature]

Сильно отклоняющиеся в положительную сторону

In [ ]:
V_k_plus

{0: ['GarageArea', 'SalePrice'],
 1: ['LotArea', 'GarageArea', 'LotFrontage'],
 2: [],
 3: [],
 4: ['SalePrice'],
 5: [],
 6: [],
 7: ['GarageArea', 'LotFrontage'],
 8: []}

Очень сильно в положительную

In [ ]:
V_k_strong_plus

{0: [],
 1: ['SalePrice'],
 2: ['LotArea', 'GarageArea', 'LotFrontage', 'SalePrice'],
 3: [],
 4: ['LotArea'],
 5: [],
 6: [],
 7: ['SalePrice'],
 8: []}

Сильно в отрицательную

In [ ]:
V_k_minus

{0: [],
 1: [],
 2: [],
 3: [],
 4: ['LotFrontage'],
 5: ['LotArea', 'GarageArea', 'SalePrice'],
 6: [],
 7: [],
 8: ['SalePrice']}

Очень сильно в отрицательную

In [ ]:
V_k_strong_minus

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}

### Описание на низком уровне

Нулевой кластер имеет сильно большие GarageArea и SalePrice, чем в среднем по датасету

Первый кластер имеет сильно большие LotArea, GarageArea, LotFrontage, чем в среднем и очень сильно больший SalePrice

Второй кластер имеет очень сильно большие все фичи

Третий кластер имеет все фичи средние по датасету

Четвертый кластер имеет SalePrice сильно больше, чем в среднем, LotArea очень сильно больше и LotFrontage сильно меньше

Пятый кластер имеет сильно меньшие LotArea, GarageArea, SalePrice

Шестой кластер имеет все на среднем уровне

Седьмой кластер имеет сильно большие GarageArea, LotFrontage, очень сильно больший SalePrice

Восьмой кластер имеет сильно меньший SalePrice

### Описание на более высоком уровне

Второй кластер имеет самые дорогие участки, 
Первый, четвертый, седьмой имеет участки примерно на следующем уровне по стоимости и размеру 

Нулевой кластер содержит участки следующего уровня, но все еще выше среднего

Третий и шестой кластера имеют средние показатели

Пятый и восьмой кластера содержат участки ниже среднего по размерам и стоимости



### Сравнение

Логично, что K-means с 9 кластерами в состоянии разбить данные на больше классов, чем с 5 кластерами.

При этом мне кажется, что 5 кластеров вполне достаточно, потому что они более наглядно показывают разницу между соответствующими кластерами

Когда мы применяем на этих данных 9 кластеров, мы разбиваем на кластеры, которые начинают дублировать друг друга, поэтому мне кажется, что столько кластеров для такой простой задачи - перебор

In [ ]:
data

,Unnamed: 0,LotArea,GarageArea,MSZoning,LotFrontage,Condition1,HouseStyle,SalePrice,YearBuilt,YrSold,Neighborhood,Label_5,Label_9
0,479,5925,672,RM,50.0,Norm,1Story,89471,1937,2007,OldTown,1,5
1,793,9158,474,RL,76.0,Norm,1Story,225000,2007,2007,Somerst,3,0
2,339,12400,297,RL,66.0,Feedr,1Story,155000,1958,2009,NAmes,2,3
3,142,8520,720,RL,71.0,Artery,1.5Fin,166000,1952,2010,NAmes,2,3
4,1453,17217,0,RL,90.0,Norm,1Story,84500,2006,2006,Mitchel,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,831,3180,480,FV,30.0,Norm,2Story,151000,2005,2006,Somerst,2,3
1456,522,5000,420,RM,50.0,Feedr,1.5Fin,159000,1947,2006,BrkSide,2,3
1457,1217,8640,552,FV,72.0,Norm,1Story,229456,2009,2009,Somerst,3,0
1458,162,12182,532,RL,95.0,Norm,1Story,220000,2005,2010,NridgHt,3,6


In [ ]:
data.to_csv('data_clustered.csv')

In [ ]:
#!cp data_clustered.csv drive/MyDrive/data_clustered.csv